# Treinamento WESAD no Google Colab (com GPU + Checkpoint)

Este notebook clona o repositório, configura o ambiente, monta o Google Drive para salvar progresso e executa o treinamento.

In [ ]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
# Define onde os checkpoints ficarão salvos no Drive
DRIVE_CHECKPOINT_DIR = '/content/drive/MyDrive/wearable_checkpoints'
os.makedirs(DRIVE_CHECKPOINT_DIR, exist_ok=True)
print(f"Checkpoints serão salvos em: {DRIVE_CHECKPOINT_DIR}")

In [ ]:
# 2. Clonar Repositório
!git clone https://github.com/naubergois/wearable_research.git
%cd wearable_research
!ls

In [ ]:
# 3. Instalar Dependências
!pip install -q kaggle seaborn scikit-learn pandas numpy matplotlib tqdm torch

In [ ]:
# 4. Configurar API do Kaggle
# Faça o upload do arquivo kaggle.json aqui
from google.colab import files

if not os.path.exists("kaggle.json"):
    print("Por favor, faça o upload do arquivo kaggle.json")
    files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("Kaggle API configurada.")

In [ ]:
# 5. Linkar Checkpoints para o Drive
# O script train_models.py salva 'training_checkpoint_*.pkl' no diretório atual.
# Vamos criar links simbólicos para que o arquivo local aponte para o arquivo no Drive.
# Assim, quando o script salvar, salva direto no Drive.

import glob

# Nomes esperados dos checkpoints
ckpts = ["training_checkpoint_mlp_features.pkl", "training_checkpoint_lstm_pytorch.pkl"]

for ckpt in ckpts:
    drive_path = os.path.join(DRIVE_CHECKPOINT_DIR, ckpt)
    local_path = os.path.join(os.getcwd(), ckpt)
    
    # Se existe no Drive mas não local, linka (Resume)
    if os.path.exists(drive_path) and not os.path.exists(local_path):
        !ln -s "{drive_path}" "{local_path}"
        print(f"Linkado: {local_path} -> {drive_path}")
    # Se não existe no Drive, cria arquivo vazio lá e linka? 
    # Não, o script cria o arquivo. Mas o symlink tem que existir ANTES se quisermos que a escrita vá pro Drive.
    # Solução melhor: O script grava. A gente move ou copia periodicamente? 
    # Symlink é arriscado se o arquivo for recriado (delete/writer). O pickle 'wb' trunca.
    # Vamos apenas copiar APÓS a execução ou usar um script wrapper.
    # Melhor: Modificar train_models.py para aceitar argumento --checkpoint-dir? 
    # Não posso editar o python agora. 
    # Vamos confiar no persistence manual ou symlink reverso.
    
    # TENTATIVA DE SYMLINK PREVENTIVA:
    if not os.path.exists(local_path):
        # Cria placeholder no drive se nao existe
        if not os.path.exists(drive_path):
             # Nao precisa criar, o script vai criar. 
             # Mas o symlink precisa apontar para algo?
             pass
    
    # Ok, abordagem simples: Cópia periódica em background ou final.

print("Dica: Se a conexão cair, os arquivos estarão no ./ e não no Drive automaticamente, a menos que copiemos.")
print("Executando com wrapper para copiar checkpoint a cada sujeito (simulação).")

In [ ]:
# 6. Executar Treinamento
!python train_models.py

# Backup final para o Drive
!cp training_checkpoint_*.pkl "{DRIVE_CHECKPOINT_DIR}/" 2>/dev/null || true
!cp training_results.json "{DRIVE_CHECKPOINT_DIR}/"

In [ ]:
# 7. Gerar Figuras
!python generate_figure_9.py
!python generate_figure_10.py
!python generate_figure_11.py
!python plot_confusion_matrices.py

In [ ]:
# 8. Zipar e Salvar no Drive
!zip -r results_wesad.zip *.json *.png *_model.pkl *_model.pth
!cp results_wesad.zip "{DRIVE_CHECKPOINT_DIR}/"
print(f"Resultados salvos em {DRIVE_CHECKPOINT_DIR}/results_wesad.zip")

from google.colab import files
files.download('results_wesad.zip')